In [1]:
import pandas as pd
import re
# from langdetect import detect, DetectorFactory  # 为什么不直接 import langdetect？？？
from tqdm import tqdm  # 进度条工具（可选）

# 歌词预处理
import nltk
from nltk.corpus import stopwords # 停用词
from nltk.tokenize import word_tokenize # 分词
from nltk.stem import PorterStemmer # 词形还原

In [2]:
# 读取文件
df = pd.read_excel(r"C:\Users\q6483\Desktop\Project\test.xlsx", engine="openpyxl")

In [3]:
# 下载 NLTK 依赖（首次运行需执行）
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('punkt_tab')

In [4]:
# 清洗无关信息
def clean_lyrics(text):

    # 检查变量是否为缺失值（NaN 或 None）
    if pd.isna(text):  
        return ""
    
    # 1.去除开头 ContributorsTranslation 信息
    text = re.sub(r"^\d+\s*Contributors?.*?Lyrics", "", text, flags=re.DOTALL | re.IGNORECASE)

    # 2.清洗广告信息（如 "See Coldplay LiveGet tickets as low as $154You might also like"）
    text = re.sub(
        r'(?si)See.*?tickets as low as \$\d+.*?You might also like',  # 匹配广告模式
        '',  # 替换为空字符串
        text)

    text = re.sub(
        r'(?im)^\s*You might also like\s*$',
        '',
        text)

    # 3.去除歌词结构标记（[Chorus]，1 ContributorWild ones (flo rida feat. sia remake) Lyrics）
    text = re.sub(r"\[.*?\]|\(.*?\)", "", text)

    # 4.去除嵌入信息
    text = re.sub(r"\d+Embed", "", text)

    # 5.去除多余空行和空格
    text = "\n".join([line.strip() for line in text.splitlines() if line.strip()])


# 预处理
    
    # 全局替换 in' → ing
    #text = re.sub(r"in'", "ing", text, flags=re.IGNORECASE)

    # 将 连字符 替换为 空格
    #text = re.sub(r"-", " ", text)
    
    # 转换为小写
    #text = text.lower()
    
    
    return text

re：
re.sub()匹配所有并替换，返回新字符串
flags=re.DOTALL 处理多行文本
flags=re.IGNORECASE 不区分大小写

^\d+   数字开头行

\s*    零个/多个空格

Contributors?  s?表示s可选

\b     单词边界，确保匹配完整的 “Contributor” 

.*?    匹配任意字符（直到遇到 Lyrics）

Lyrics 匹配结尾的 Lyrics

如果不处理缺失值，可能会导致以下问题：

程序崩溃：某些操作（如字符串操作）无法处理 NaN 或 None。

数据不一致：缺失值可能导致后续分析或清洗逻辑出错。

AttributeError: 'NoneType' object has no attribute 'strip'

In [8]:
# 歌词处理
def process_lyrics(input_file, output_file):
    
    # 清洗歌词
    tqdm.pandas(desc="Cleaning Lyrics")  #
    df["lyrics"] = df["lyrics"].progress_apply(clean_lyrics) # .apply(): pandas的方法，对 DataFrame 的某一列（或行）应用一个函数
                                                             # .progress_apply(): tqdm库 进度条功能
    
    # 保存结果（注意：tokens 列在 Excel 中会显示为字符串列表）
    df.to_excel(output_file, index=False)
    
    print(f"清洗完成，结果已保存至{output_file}")

In [9]:
input_file = r"C:\Users\q6483\Desktop\Project\test.xlsx"
output_file = r"C:\Users\q6483\Desktop\Project\test1.xlsx"

# 运行函数
process_lyrics(input_file, output_file)

Cleaning Lyrics: 100%|█████████████████████████████████████████████████████████████| 594/594 [00:00<00:00, 1406.65it/s]


清洗完成，结果已保存至C:\Users\q6483\Desktop\Project\test1.xlsx
